# `1 Table formatting`

<ol start=0>
    <li>Libraries and constants</li>
    <li>Add description to the columns</li>
    <li>Dtypes</li>
<ol>

## 1.0 Libraries and constants

In [1]:
# Libraries
import pandas as pd

# Project utils
from utils.data_utils   import map_column_inplace
from utils.memory       import MemoryUsageTracker
from utils.data_utils   import column_map

In [2]:
# Constants
COLUMNS_PATH                = "data/metadata/columns.csv"
COLUMNS                     = pd.read_csv(COLUMNS_PATH, index_col=0)

DTYPES                      = COLUMNS.dtype

CLASSES_PATH                = "data/metadata/classes.csv"
CLASSES                     = pd.read_csv(CLASSES_PATH, index_col=0)

GERMAN_CREDIT_DATA_PATH     = "data/statlog+german+credit+data/german.data"
GERMAN_CREDIT_DATA          = pd.read_csv(GERMAN_CREDIT_DATA_PATH, sep=' ', header=None)

## 1.1 Add description to the columns

In [3]:
# Preserve the original data
german_credit_data = GERMAN_CREDIT_DATA.copy()

In [4]:
# Update column names with COLUMNS.index
german_credit_data.columns = COLUMNS.index

In [5]:
# Column names
for column in german_credit_data.columns:
    print(column)

A1 - Status of existing checking account
A2 - Duration in month
A3 - Credit history
A4 - Purpose
A5 - Credit amount
A6 - Savings account/bonds
A7 - Present employment since
A8 - Installment rate in percentage of disposable income
A9 - Personal status and sex
A10 - Other debtors/guarantors
A11 - Present residence since
A12 - Property
A13 - Age in years
A14 - Other installment plans
A15 - Housing
A16 - Number of existing credits at this bank
A17 - Job
A18 - Number of people being liable to provide maintenance for
A19 - Telephone
A20 - Foreign worker
T1 - Is good credit


## 1.2 Dtypes

In [6]:
# Track dtypes evolution
dtypes_evolution = pd.DataFrame()
dtypes_evolution = pd.concat([dtypes_evolution, german_credit_data.dtypes], axis=1)

In [7]:
# Track memory usage at first
memory_tracker = MemoryUsageTracker(german_credit_data.memory_usage(deep=True).sum())

In [8]:
# Dtypes before conversion
boolean_columns = ['A19 - Telephone',
                   'A20 - Foreign worker',
                   'T1 - Is good credit']

german_credit_data[boolean_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   A19 - Telephone       1000 non-null   object
 1   A20 - Foreign worker  1000 non-null   object
 2   T1 - Is good credit   1000 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 23.6+ KB


In [9]:
from utils.data_utils import column_map

In [10]:
# Map boolean columns
for (column, boolean_map) in column_map.items():
    map_column_inplace(
        df          = german_credit_data,
        column      = column,
        value_map   = boolean_map
    )

In [11]:
# Dtypes after conversion
german_credit_data[boolean_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   A19 - Telephone       1000 non-null   bool 
 1   A20 - Foreign worker  1000 non-null   bool 
 2   T1 - Is good credit   1000 non-null   bool 
dtypes: bool(3)
memory usage: 3.1 KB


In [12]:
## 2 Convert dtypes ----------------------------------------------------------------------------------------------
for column in german_credit_data.columns:
    german_credit_data[column] = german_credit_data[column].astype(DTYPES[column])

In [13]:
# Append dtypes evolution
dtypes_evolution = pd.concat([dtypes_evolution, german_credit_data.dtypes], axis=1)

# Name columns of dtypes_evolution
dtypes_evolution.columns = ['Initial', 'After mapping']

In [14]:
dtypes_evolution[where_changed := dtypes_evolution['Initial'] != dtypes_evolution['After mapping']]

,Initial,After mapping
A1 - Status of existing checking account,object,category
A3 - Credit history,object,category
A4 - Purpose,object,category
A6 - Savings account/bonds,object,category
A7 - Present employment since,object,category
A8 - Installment rate in percentage of disposable income,int64,category
A9 - Personal status and sex,object,category
A10 - Other debtors/guarantors,object,category
A12 - Property,object,category
A14 - Other installment plans,object,category


In [15]:
# Update memory usage tracker
memory_tracker.update(german_credit_data.memory_usage(deep=True).sum())

# Visualize memory usage as a DataFrame
memory_usage_df = pd.DataFrame({
    'Previous usage (bytes)': [memory_tracker.previous],
    'Current usage (bytes)': [memory_tracker.latest],
    'Difference (bytes)': [memory_tracker.diff],
    'Difference (%)': [(memory_tracker.diff / memory_tracker.previous) * 100 if memory_tracker.previous else None]
})

memory_usage_df = memory_usage_df.T
memory_usage_df.columns = ['Memory Usage']

In [16]:
memory_usage_df.head()

,Memory Usage
Previous usage (bytes),747144.000000
Current usage (bytes),67782.000000
Difference (bytes),-679362.000000
Difference (%),-90.927853


# TEST

In [17]:
# Import
from tests.table_formatting_test import TableFormattingTest

In [18]:
test = TableFormattingTest()
test.test_column_statistics(
    original = GERMAN_CREDIT_DATA,
    processed = german_credit_data
)

All column statistics are unchanged after processing.


# END

In [19]:
# Final German Credit Data
german_credit_data.head()

label,A1 - Status of existing checking account,A2 - Duration in month,A3 - Credit history,A4 - Purpose,A5 - Credit amount,A6 - Savings account/bonds,A7 - Present employment since,A8 - Installment rate in percentage of disposable income,A9 - Personal status and sex,A10 - Other debtors/guarantors,...,A12 - Property,A13 - Age in years,A14 - Other installment plans,A15 - Housing,A16 - Number of existing credits at this bank,A17 - Job,A18 - Number of people being liable to provide maintenance for,A19 - Telephone,A20 - Foreign worker,T1 - Is good credit
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,True,True,True
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,False,True,False
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,False,True,True
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,False,True,True
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,False,True,False


In [20]:
# Save Formatted Table
german_credit_data.to_csv('data/processed-data/1-formatted-table.csv', index=False)